In [349]:
import pandas as pd
from collections import Counter

In [350]:
filename = "C:\\Users\\piete\\AppData\\Roaming\\MobaXterm\\slash\\RemoteFiles\\465348_2_1\\2020-2022_BAK Cakes&Sweet_Goods_WE1 (8300).xls"

In [351]:
df = pd.read_excel(filename, usecols=["Product Id", "Product Name", "Claims/Features"])

In [352]:
df = df.dropna(subset=["Claims/Features"])

In [353]:
df["Claims/Features"] = df["Claims/Features"].str.lower()
df["Claims/Features"] = df["Claims/Features"].str.replace(",", ".").str.replace("with ", "").str.replace("without", "no")

In [354]:
#remove last dot in each claims/features. then turn the claims/features into a list by splitting the content by the dots
df["Claims/Features"] = df["Claims/Features"].str.rstrip(".").str.split("\. ")

In [355]:
claims = []
for list in df["Claims/Features"]:
    for claim in list:
        claims.append(claim)

In [356]:
Claim_ammount = Counter(claims)

In [357]:
Claim_ammount.most_common()

[('suitable for vegetarians', 1162),
 ('green dot certified', 974),
 ('recyclable packaging', 769),
 ('recyclable', 650),
 ('christmas', 421),
 ('gluten free', 331),
 ('this is a microwaveable product', 311),
 ('palm oil free', 300),
 ('cocoa certified', 237),
 ('rainforest alliance people and nature', 235),
 ('nutriscore d', 220),
 ('tray recycle', 191),
 ('no preservatives', 177),
 ('fsc certified', 161),
 ('suitable for vegans', 137),
 ('lactose free', 136),
 ('box recycle', 129),
 ('no artificial colors', 121),
 ('christmas edition', 118),
 ('certified organic', 116),
 ('carton recycle', 116),
 ('nutriscore e', 110),
 ('certified ab organic', 97),
 ('rainforest alliance certified cocoa', 93),
 ('no palm oil', 79),
 ('flavors or hydrogenated fat', 74),
 ('fairtrade cocoa', 73),
 ('free range eggs', 68),
 ('sleeve recycle', 64),
 ('no artificial colors or flavors', 63),
 ('wheat free', 56),
 ('certified halal', 56),
 ('nutriscore c', 53),
 ('no hydrogenated fats', 53),
 ('people and 

first make manual list


#bert


NMF unsupervised clustering method


TF-IDF

Manual list:
vegetarian
green dot certified
recyclable
christmas
easter
valentine's day
king's day
gluten free
microwaveable
palm oil free
cocoa certified
nutriscore c
nutriscore d
nutriscore e
gmo free
fsc certified
vegan
lactose free
no preservatives
no artificial colors
no artificial flavors
certified organic
certified halal
fairtrade cocoa
free range eggs
milk free

In [1]:
manual_list = ["vegetarian",
"green dot certified",
"recyclable",
"christmas",
"easter",
"valentine's day",
"king's day",
"gluten free",
"microwaveable",
"palm oil free",
"cocoa certified",
"nutriscore c",
"nutriscore d",
"nutriscore e",
"gmo free",
"fsc certified",
"vegan",
"lactose free",
"no preservatives",
"no artificial colors",
"no artificial flavors",
"certified organic",
"certified halal",
"fairtrade cocoa",
"free range eggs",
"milk free"]

In [2]:
manual_list

['vegetarian',
 'green dot certified',
 'recyclable',
 'christmas',
 'easter',
 "valentine's day",
 "king's day",
 'gluten free',
 'microwaveable',
 'palm oil free',
 'cocoa certified',
 'nutriscore c',
 'nutriscore d',
 'nutriscore e',
 'gmo free',
 'fsc certified',
 'vegan',
 'lactose free',
 'no preservatives',
 'no artificial colors',
 'no artificial flavors',
 'certified organic',
 'certified halal',
 'fairtrade cocoa',
 'free range eggs',
 'milk free']